In [1]:
import numpy as np
import pandas as pd
import os

import torch
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import Dataset, TensorDataset

from bertviz import model_view
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, PretrainedConfig, BertPreTrainedModel

I0114 13:56:10.141602 140734947571136 file_utils.py:35] PyTorch version 1.3.1 available.
I0114 13:56:12.728013 140734947571136 file_utils.py:48] TensorFlow version 2.0.0 available.


In [2]:
class BertMultiHeadModel(BertPreTrainedModel): # FAKE SEQUENCE MODEL
    def __init__(self, config):
        super(BertMultiHeadModel, self).__init__(config)
        self.num_labels = [2, 4] # ignore config.num_labels # should be a list!
        self.num_tasks = 2 # CUSTOM EDIT: MANUALLY SPECIFIED NUM_TASKS
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = [nn.Linear(config.hidden_size, self.num_labels[i]) for i in range(self.num_tasks)]
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
        task = 1 # 0 for wiki, 1 for fake news
        if type(task) != int:
            raise Exception("BertMulti model first input must be task index (int)!")
        
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier[task](pooled_output) # CUSTOM EDIT: specify which linear layer with task index
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels[task]), labels.view(-1))
            outputs = (loss,) + outputs
        return outputs  # (loss), logits, (hidden_states), (attentions)

In [3]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/5.7.0/d3.min',
    jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

# Load data

In [4]:
# fake news
body_df = pd.read_csv("../nlp-final-project/data/fake_news_bodies.csv")
stance_df = pd.read_csv("../nlp-final-project/data/fake_news_stances.csv")
stance_to_idx = {}
stances = stance_df["Stance"].drop_duplicates().values
for i, stance in enumerate(stances):
    stance_to_idx[stance] = i
num_stances = len(stance_to_idx)

x_list = []
y_list = []
idx_to_id = {body_id:i for (i, body_id) in enumerate(body_df['Body ID'])}

for body_id, headline, stance in zip(stance_df["Body ID"], stance_df["Headline"], stance_df["Stance"]):
    body = body_df.iloc[idx_to_id[body_id]]["articleBody"]
    x_list.append(headline + " [SEP] " + body)
    y_list.append(stance_to_idx[stance])

In [5]:
print(stance_df['Stance'].unique())

['unrelated' 'agree' 'disagree' 'discuss']


In [33]:
# wiki
cutoff = 0.3
comment_df = pd.read_csv("../nlp-final-project/data/attack_annotated_comments.tsv", sep ='\t')
comment_df = comment_df.drop(columns=['logged_in', 'ns', 'sample'])
comment_df["comment"] = comment_df["comment"].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comment_df["comment"] = comment_df["comment"].apply(lambda x: x.replace("TAB_TOKEN", " "))

annotation_df = pd.read_csv("../nlp-final-project/data/attack_annotations.tsv",  sep='\t')
annotation_df = (annotation_df.groupby("rev_id")["attack"].mean() > cutoff)
annotation_df = annotation_df.to_frame().reset_index()
final_df = pd.merge(comment_df, annotation_df, how='inner', on=['rev_id'])

# Load multi-head model

In [6]:
do_lower_case = True
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=do_lower_case)

I0114 13:56:25.197524 140734947571136 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/johnhallman/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [8]:
model_version = "../nlp-final-project/multi-epoch-11/pytorch_model.bin"
config_name = PretrainedConfig().from_json_file("../nlp-final-project/multi-epoch-11/config.json")
multi_model = BertMultiHeadModel.from_pretrained(model_version, config=config_name)

I0114 14:04:28.233350 140734947571136 modeling_utils.py:403] loading weights file ../nlp-final-project/multi-epoch-11/pytorch_model.bin


# Load single-head model (both datasets)

In [15]:
model_version = "../nlp-final-project/fake-epoch-10/pytorch_model.bin"
config_name = PretrainedConfig().from_json_file("../nlp-final-project/fake-epoch-10/config.json")
fake_model = BertForSequenceClassification.from_pretrained(model_version, config=config_name)

I0114 14:05:21.479539 140734947571136 modeling_utils.py:403] loading weights file ../nlp-final-project/fake-epoch-10/pytorch_model.bin


In [16]:
model_version = "../nlp-final-project/wiki-epoch-10/pytorch_model.bin"
config_name = PretrainedConfig().from_json_file("../nlp-final-project/wiki-epoch-10/config.json")
wiki_model = BertForSequenceClassification.from_pretrained(model_version, config=config_name)

I0114 14:05:33.501743 140734947571136 modeling_utils.py:403] loading weights file ../nlp-final-project/wiki-epoch-10/pytorch_model.bin


# Load untrained model

In [10]:
untrained_model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', config=config_name)

I0114 14:04:34.272482 140734947571136 modeling_utils.py:406] loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin from cache at /Users/johnhallman/.cache/torch/transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
I0114 14:04:37.905997 140734947571136 modeling_utils.py:480] Weights of BertForSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
I0114 14:04:37.909646 140734947571136 modeling_utils.py:483] Weights from pretrained model not used in BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']


# Text classification

In [11]:
def classify(model, sentence):
    output = model(torch.tensor(tokenizer.encode(sentence, pad_to_max_length=True, max_length=512)).unsqueeze(0)) # head = 1
    print(int(torch.argmax(output[0], axis=1)[0]), " - ", output[0])
    return torch.argmax(output[0], axis=1)

In [12]:
print("fake")
for i in range(2):
    classify(fake_model, x_list[i])
    print(y_list[i])
print("multi")
for i in range(2):
    classify(multi_model, x_list[i])
    print(y_list[i])

single
0  -  tensor([[ 3.9179, -0.7485, -2.7601, -0.2044]], grad_fn=<AddmmBackward>)
0
1  -  tensor([[-3.0004,  1.4779,  0.1758,  0.9192]], grad_fn=<AddmmBackward>)
1
multi
2  -  tensor([[-0.3015,  0.0591,  0.3105, -0.6409]], grad_fn=<AddmmBackward>)
0
3  -  tensor([[-0.5599, -0.6209, -0.7238,  0.2603]], grad_fn=<AddmmBackward>)
1


In [27]:
def concat(s_list):
    s = ""
    for i in s_list: s += i + " "
    return s[:-1]

def get_sentences(i, max_length=32):
    s = concat(x_list[i].split()[:max_length])
    try:
        s_a, s_b = s.split('[SEP]')[0], s.split('[SEP]')[1]
    except Exception as e:
        #print(s)
        s, s_a, s_b = None, None, None
    return s, s_a, s_b

# Visualization code

In [13]:
def show_model_view(model, tokenizer, sentence_a, sentence_b=None, hide_delimiter_attn=False):
    inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
    token_type_ids = inputs['token_type_ids']
    input_ids = inputs['input_ids']
    attention = model(input_ids, token_type_ids=token_type_ids)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    if sentence_b:
        sentence_b_start = token_type_ids[0].tolist().index(1)
    else:
        sentence_b_start = None
    if hide_delimiter_attn:
        for i, t in enumerate(tokens):
            if t in ("[SEP]", "[CLS]"):
                for layer_attn in attention:
                    layer_attn[0, :, i, :] = 0
                    layer_attn[0, :, :, i] = 0
    model_view(attention, tokens, sentence_b_start)

In [76]:
count = 0
for i in range(10000, 100000):
    s = final_df['comment'].iloc[i]
    if len(s.split()) < 50 and final_df['attack'].iloc[i]:
        count += 1
        print(s)
        print()
        if count == 20:
            break

  == Tom Likes it Up the Pooper!!! ==  BY MEN!!!

  == About the personal attacks and vandalyzing ==  I appologise about the personal attacks. I usualy don't do that.  About vandalizing: see what user Nikosilver is doing - he is the real vandalizer, not me. Please have him warned.

  ==Personal attack?== I just pointed out he's being a WP:DICK. Why can he call multiple users dicks but we can't return the favor? Talk about a one sided policy.  

` *Calling someone a ``cocksucker`` is a violation of WP:NPA, and it can get you blocked.  Don't do it, ne? - `

hope you ROT IN HELL YOU MOTHER FUCKING IDIOT! PERISH IN THE FLAMING BOWELS OF HELL! I HOP THAT YOU DIE ALONE, AND SAD you FUCKER! I NEED YOU TO PERISH! END UR LIFE RIGHT NOW

comment this page is dumb and should be removed    

  == Ccwaters is still a fuck ==  You're still a fuck, good luck on being a worldwide fuck

  == WikiDicks ==  Nothing any of you old scholarly wannabe fucks can say about my article will negatively affect me 

In [84]:
#model = untrained_model
#model = fake_model
#model = wiki_model
model = multi_model

s = " == hey idiot ==  you were the one that was repeatedly undoing MY edits. Go away, AND STOP VANDALISING!"

#s, s_a, s_b = get_sentences(320)

#print(s_a)
#print("\n---------------\n")
#print(s_b)

#s_a = "this guy is clearly a moron"
#s_b = "you are an idiot"
#s = s_a + " [SEP] " + s_b
#show_model_view(model, tokenizer, s_a, s_b, hide_delimiter_attn=False)
show_model_view(model, tokenizer, s, hide_delimiter_attn=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>